In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import pickle

from loading_data import get_data
from plot_behavior import combine_behavior
from analyze_tuning_curves import get_tuning_curves
from analyze_decode import get_decoded, zone_sort, point_in_zones

In [ ]:
import info.r066d5 as info

In [ ]:
from loading_data import load_data

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "decode_wout_trial")

In [ ]:
from analyze_decode_bytrial import get_trials, decode_trial

In [ ]:
# events, position, spikes, lfp, lfp_theta = get_data(info)
events, position, spikes, lfp_swr, lfp_theta = load_data(info)

position = position.time_slice(info.task_times['phase3'].start, info.task_times['phase3'].stop)
spikes = [spiketrain.time_slice(info.task_times['phase3'].start, info.task_times['phase3'].stop) for spiketrain in spikes]

xedges, yedges = nept.get_xyedges(position)

trial_epochs = get_trials(events, info.task_times['phase3'])

In [ ]:
plt.plot(position.time, position.y, 'b.')
plt.show()

In [ ]:
position.n_samples

In [ ]:
%%time
all_trials = []
shuffled = False

for this_trial in range(trial_epochs.n_epochs):
# for this_trial in [1]:
    print("trial:", this_trial)
    
    trial_start = trial_epochs.starts[this_trial]
    trial_stop = trial_epochs.stops[this_trial]
    
    trial_times = nept.Epoch([trial_start, trial_stop])
    trial_number = "test"
    neurons = get_tuning_curves(info, position, spikes, xedges, yedges, speed_limit=0.4, phase_id="phase3", trial_times=trial_times, trial_number=trial_number, cache=False)
    
    decode = decode_trial(info, neurons, trial_times, this_trial, shuffled)
    
    all_trials.append(decode)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)

error_byactual_position = np.zeros((len(yedges), len(xedges)))
n_byactual_position = np.ones((len(yedges), len(xedges)))

for idx, decode in enumerate(all_trials):
    for error, x, y in zip(decode['errors'], decode['position'].x, decode['position'].y):
        x_idx = nept.find_nearest_idx(xedges, x)
        y_idx = nept.find_nearest_idx(yedges, y)
        error_byactual_position[y_idx][x_idx] += error
        n_byactual_position[y_idx][x_idx] += 1

    error_byactual = error_byactual_position / n_byactual_position
    
print("error")
pp = plt.pcolormesh(xx, yy, error_byactual, vmin=0., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, "decoding_wout_current_trial-" + info.session_id + "-error.png"))

In [ ]:
print("position occupancy")
pp = plt.pcolormesh(xx, yy, n_byactual_position, vmin=0., vmax=1000., cmap="pink_r")
plt.colorbar(pp)
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, "decoding_wout_current_trial-" + info.session_id + "-occupancy.png"))

In [ ]:
print("tuning curves")
xx, yy = np.meshgrid(xedges, yedges)

all_tuning_curves = np.zeros(neurons.tuning_shape)
for i in range(neurons.n_neurons):
    all_tuning_curves += neurons.tuning_curves[i]

pp = plt.pcolormesh(xx, yy, all_tuning_curves, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
# idx = 31
for i, (start, stop) in enumerate(zip(trial_epochs.starts, trial_epochs.stops)):
    print(i)
    pos = position.time_slice(start, stop)
    
    plt.plot(pos.x, pos.y, 'g.')
    plt.plot(pos.x[-5:], pos.y[-5:], 'r.')
    plt.plot(pos.x[:5], pos.y[:5], 'c.')
    plt.show()

In [ ]:
plt.plot(decode['position'].x, decode['position'].y, 'g.')
plt.show()

In [ ]:
plt.plot(position.time, position.x, 'b.')
plt.show()

In [ ]:
for decode in all_trials:
    print("decoded")
    xx, yy = np.meshgrid(xedges, yedges)
    histogram, xs, ys = np.histogram2d(decode['decoded'].x, decode['decoded'].y, bins=xx.shape)

    cmap = plt.cm.get_cmap('bone_r')
    pp = plt.pcolormesh(yy, xx, histogram, vmin=0., vmax=800., cmap=cmap)
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()
    
    print("position")
    xx, yy = np.meshgrid(xedges, yedges)
    histogram, xs, ys = np.histogram2d(decode['position'].x, decode['position'].y, bins=xx.shape)

    cmap = plt.cm.get_cmap("pink_r")
    pp = plt.pcolormesh(yy, xx, histogram, vmin=0., vmax=50., cmap=cmap)
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
for decode in all_trials:
    plt.plot(decode['position'].x, decode['position'].y, '.')
plt.show()

In [ ]:
for decode in all_trials:
    plt.plot(decode['decoded'][59:63].x, decode['decoded'][59:63].y, '.')
plt.show()

In [ ]:
decode.keys()

In [ ]:
plt.plot(decode['decoded'].x, decode['decoded'].y, 'k.')
plt.plot(decode['position'].x, decode['position'].y, 'b.', ms=1)
plt.show()

In [ ]:
print("decoded")
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decode['decoded'].x, decode['decoded'].y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r')
pp = plt.pcolormesh(yy, xx, histogram, vmin=0., vmax=800., cmap=cmap)
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
print("position")
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decode['position'].x, decode['position'].y, bins=xx.shape)

cmap = plt.cm.get_cmap("pink_r")
pp = plt.pcolormesh(yy, xx, histogram, vmin=0., vmax=50., cmap=cmap)
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
error_hist_byactual = np.zeros((len(yedges), len(xedges)))
n_hist_byactual = np.ones((len(yedges), len(xedges)))
error_x = []
error_y = []

for error, x, y in zip(decode['errors'], position.x, position.y):
    x_idx = nept.find_nearest_idx(xedges, x)
    y_idx = nept.find_nearest_idx(yedges, y)
    error_hist_byactual[y_idx][x_idx] += error
    n_hist_byactual[y_idx][x_idx] += 1

error_byactual = error_hist_byactual / n_hist_byactual

In [ ]:
pp = plt.pcolormesh(xx, yy, error_byactual, cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, n_hist_byactual, vmin=0., vmax=50., cmap="pink_r")
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, error_hist_byactual, vmin=0., vmax=50., cmap="pink_r")
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import pickle

from loading_data import get_data
from plot_behavior import combine_behavior
from analyze_tuning_curves import get_tuning_curves
from analyze_decode import get_decoded, zone_sort, point_in_zones

import info.r063d2 as info

events, position, spikes, lfp, lfp_theta = get_data(info)

In [ ]:
from startup import load_shortcut_position
from loading_data import unzip_nvt_file

thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
position_path = os.path.join(dataloc, 'data-working', info.rat_id, info.session+'_recording')
unzip_nvt_file(position_path, info.session+'-VT1')
position = load_shortcut_position(info, os.path.join(dataloc, info.position_filename), events)
os.remove(os.path.join(position_path, info.session+'-VT1.nvt'))

In [ ]:
plt.plot(position.time, position.x, 'k.')
plt.show()

In [ ]:
plt.plot(position.time, position.y, 'm.')
plt.show()

In [ ]:
position.n_samples

In [ ]:
# time_thresh=1.0, jump_thresh=20, dist_thresh=5 -> 22518
# time_thresh=1.0, jump_thresh=10, dist_thresh=5 -> 225113
# time_thresh=1.0, jump_thresh=10, dist_thresh=2 -> 236120
# time_thresh=1.0, jump_thresh=10, dist_thresh=1 -> 237942
# time_thresh=1.0, jump_thresh=10, dist_thresh=3 -> 234790
# time_thresh=1.0, jump_thresh=20, dist_thresh=3 -> 234795
# time_thresh=1.0, jump_thresh=20, dist_thresh=2 -> 236125 *
# time_thresh=1.0, jump_thresh=30, dist_thresh=2 -> 
position.n_samples 